In [2]:
#please ensure to run all the cells in order to have the maps fully function!
import csv
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap
import bqplot
from bqplot import pyplot as plt
from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Scale, LogScale
from ipyleaflet import Map, GeoJSON, WidgetControl, SearchControl, LayerGroup, AwesomeIcon, Marker, Circle, FullScreenControl, LayersControl, basemaps
import os
import json
from ipywidgets import interact, interact_manual

ModuleNotFoundError: No module named 'folium'

In [121]:
def CSVtoLIST(file):
    dataList = []
    with open(file, newline='') as dataFile:
        csvReader = csv.reader(dataFile)
        next(csvReader, None)
        for line in csvReader:
            if line[0] == 'NA' or line[1] == 'NA':
                continue
            else:
                tempList = []
                tempList.insert(len(tempList), float(line[0]))
                tempList.insert(len(tempList), float(line[1]))
                tempList.insert(len(tempList), line[2])
                tempList.insert(len(tempList), line[3])
                tempList.insert(len(tempList), line[4])
                tempList.insert(len(tempList), line[5])
                dataList.insert(len(dataList), tempList)
    return dataList

def CSVtoListPrice(file):
    listWithPrices = []
    with open(file, newline='') as dataFile:
        csvReader = csv.reader(dataFile)
        next(csvReader, None)
        for line in csvReader:
            if line[0] == 'NA' or line[1] == 'NA':
                continue
            else:
                tempList = []
                tempList.insert(len(tempList), float(line[0]))
                tempList.insert(len(tempList), float(line[1]))
                if line[2] == 'NA':
                    tempList.insert(len(tempList), float(-1))
                else:
                    tempList.insert(len(tempList), float(line[2]))
                tempList.insert(len(tempList), line[3])
                tempList.insert(len(tempList), line[4])
                tempList.insert(len(tempList), line[5])
                listWithPrices.insert(len(listWithPrices), tempList)
    return listWithPrices

def listToDataFrame(dataList):
    dataDict = {'Lat': [], 'Lon': [], 'Infl_Price': [], 'Date': [], 'Neighborhood': [], 'DT': [], 'Year': []}
    for data in dataList:
        dataDict["Lat"].insert(len(dataDict["Lat"]), data[0])
        dataDict["Lon"].insert(len(dataDict["Lon"]), data[1])
        dataDict["Infl_Price"].insert(len(dataDict["Infl_Price"]), data[2])
        dataDict["Date"].insert(len(dataDict["Date"]), data[3])
        dataDict["Neighborhood"].insert(len(dataDict["Neighborhood"]), data[4])
        dataDict["DT"].insert(len(dataDict["DT"]), data[5])
        dataDict['Year'].insert(len(dataDict['Year']), int(data[3][0:4]))
    return dataDict

In [124]:
#create dataframe for home price figure
data = listToDataFrame(CSVtoLIST('Cleaned_Housing_Data.csv'))
df = pd.DataFrame(data=data)
priceData = listToDataFrame(CSVtoListPrice('Cleaned_Housing_Data.csv'))
priceDF = pd.DataFrame(data=priceData)

#create dataframe for fruitbelt figure priming
chartData = priceDF[(priceDF['Infl_Price'] > 1) & (priceDF['Year'] < 2021)]
priceData = chartData.groupby(['Neighborhood', 'Year']).Infl_Price.mean().reset_index()
priceDataFB = priceData[(priceData['Neighborhood'] == 'Fruit Belt') & (priceData['Year'] < 2021)]

#create dataframe for foreclosures figure
foreclosures = chartData[(chartData['DT'] == 'D1D') & (df['Year'] <= 2020)]
foreclosures = foreclosures.groupby(['Neighborhood', 'Year']).size().reset_index()
foreclosures = foreclosures.rename(columns={0: 'Foreclosures'})
foreclosuresFB = foreclosures[(foreclosures['Neighborhood'] == 'Fruit Belt')]

#create dataframe for onlick function for prediction data
predictionData = pd.read_csv('prediction_Data.csv')
predictionDataDict = []
for row in predictionData.itertuples():
    tempDict = {'SBL': row[1],
                'ProbSell': row[2],
                'Neighborhood': row[3],
                'Lat': row[8],
                'Lon': row[9],
                'Bought': row[10],
                'ProbBought': row[11],
                'ProbNotBought': row[12]
    }
    predictionDataDict.insert(len(tempDict), tempDict)

C:\Users\KevinBrito\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [125]:
#Creation of average home price figure/plot
date_start = pd.datetime(1985, 12, 31)
date_end = pd.datetime(2020, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)
date_data = pd.date_range(start=date_start, end=date_end, freq='A', normalize=True)

x_scale = LinearScale()

lines = Lines(x=priceDataFB.Year, y=priceDataFB.Infl_Price, scales={'x': LinearScale(), 'y': x_scale})
ax_x = Axis(label='Year', scale=date_scale, num_ticks=7, tick_format='%Y')
ax_y = Axis(label='Price', scale=x_scale, orientation='vertical', side='left', label_offset='50px')
averageHomePricePlot = Figure(axes=[ax_x, ax_y], title='Average Home Price in Fruit Belt', marks=[lines], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '700px'})

averageHomePricePlot

Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=7, scale=DateScale(max=datetime.datetime(202…

In [128]:
#Find a start date and end date
date_start = pd.datetime(1994, 12, 31)
date_end = pd.datetime(2011, 12, 31)

#Find a scale for the data
date_scale = DateScale(min=date_start, max=date_end)
date_data = pd.date_range(start=date_start, end=date_end, freq='A', normalize=True)

x_scale = LinearScale()
foreclosureLines = Lines(x=foreclosuresFB.Year, y=foreclosuresFB.Foreclosures, scales={'x': LinearScale(), 'y': x_scale})
foreclosureAx_x = Axis(label='Year', scale=date_scale, num_ticks=7, tick_format='%Y')
foreclosureAx_y = Axis(label='Number of Foreclosures', scale=x_scale, orientation='vertical', side='left')
foreclosurePlot = Figure(axes=[foreclosureAx_x, foreclosureAx_y], title='Number of Foreclosures in Fruit Belt', marks=[foreclosureLines], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '700px'})

foreclosurePlot

Figure(animation_duration=500, axes=[Axis(label='Year', num_ticks=7, scale=DateScale(max=datetime.datetime(201…

In [129]:
def update_homePrice(neighborhood_name, data):
    lines.y = data[(data['Neighborhood'] == neighborhood_name) & (data['Year'] < 2021)].Infl_Price
    ax_y.label = 'Price'
    averageHomePricePlot.title = "Average Home Price in " + neighborhood_name

update_homePrice('Allentown', priceData)

def update_foreclosures(neighborhood_name, data):
    foreclosureLines.y = data[(data['Neighborhood'] == neighborhood_name)].Foreclosures
    foreclosureLines.x = data[(data['Neighborhood'] == neighborhood_name)].Year
    foreclosureAx_y = "Foreclosures"
    foreclosurePlot.title = "Foreclosures in " + neighborhood_name

update_foreclosures('Allentown', foreclosures)

In [130]:
#please use this cell to demonstrate on hover and on click effects!
center = [42.880230, -78.878738]
dynamicMap = Map(center=center, zoom=11)

marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

search = SearchControl(position="topright", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=5,
                       property_name='display_name',
                       marker=marker
                      )

dynamicMap.add_control(search)

widget_control1 = WidgetControl(widget=averageHomePricePlot, position='bottomleft')

dynamicMap.add_control(widget_control1)

with open('Neighborhoods.geojson') as f:
    neighborhoods = json.load(f)
    
geo = GeoJSON(data=neighborhoods, style={'fillColor': 'orange', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
dynamicMap.add_layer(geo)


control = LayersControl(position='bottomright')

def onHoverHomePrice(event, feature, **kwargs):
    neighborhood = feature['properties']['nbhdname']
    update_homePrice(neighborhood, priceData)
    
def onHoverForeclosures(event, feature, **kwargs):
    neighborhood = feature['properties']['nbhdname']
    update_foreclosures(neighborhood, foreclosures)
            
def on_click(event, feature, **kwargs):
    neighborhood = feature['properties']['nbhdname']
    for index in range(0, len(predictionDataDict)):
        if predictionDataDict[index]['Neighborhood'] == neighborhood:
            circle = Circle()
            circle.color = "blue"
            circle.fill_color = "yellow"
            circle.location = (predictionDataDict[index]['Lat'], predictionDataDict[index]['Lon'])
            circle.radius = 10
            if predictionDataDict[index]['Bought'] == 1:
                circle.color = "green"
            else:
                circle.color = "red"
            if predictionDataDict[index]['ProbBought'] >= 0.5:
                circle.fill_color = "blue"
            else:
                circle.fill_color = "orange"
            basicMapGeoData.add_layer(circle)
            
            
dynamicMap.add_control(FullScreenControl())
dynamicMap.add_control(control)

                
geo.on_hover(onHoverHomePrice)
geo.on_click(on_click)
dynamicMap

Map(center=[42.88023, -78.878738], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [1]:
#please use this cell to demonstrate example control of 
with open('Neighborhoods.geojson') as f:
    neighborhoods = json.load(f)

basicMap = Map(center = [42.880230, -78.878738], zoom = 11)
background = basemaps.OpenStreetMap.Mapnik
basicMap.add_control(FullScreenControl())
basicMapGeoData = GeoJSON(data=neighborhoods, style={'fillColor': 'orange', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Neighborhoods')
layers = []
widgets = []
plotDict = {'Average Home Price': averageHomePricePlot, 'Foreclosure': foreclosurePlot}
dataDict = {'Average Home Price': priceData, 'Foreclosure': foreclosures}

@interact
def changeMap(Maps=['None','Basic Map', 'Clear']):
    if Maps == 'Clear':
        if layers != []:
            basicMap.clear_layers()
            layers.clear()
        basicMap.add_layer(background)
        layers.append('Background')
    elif Maps == 'Basic Map':
        if 'Geo' in layers:
            basicMap.remove_layer(basicMapGeoData)
        basicMap.add_layer(basicMapGeoData)
        layers.append('Geo')
        
@interact
def changeFigure(Figures=['None', 'Average Home Price', 'Foreclosure', 'Clear']):
    global plotWidget
    if Figures != 'Clear' and Figures != 'None':
        if 'Geo' in layers:
            if len(widgets) == 0:
                plotWidget = WidgetControl(widget=plotDict[Figures], position='bottomleft', max_width=700, max_height=700)
                basicMap.add_control(plotWidget)
                widgets.append(plotDict[Figures])
    elif Figures == 'Clear':
        basicMap.remove_control(plotWidget)
        widgets.clear()
    if Figures == 'Average Home Price':
                    basicMapGeoData.on_hover(onHoverHomePrice)
    if Figures == 'Foreclosure':
                    basicMapGeoData.on_hover(onHoverForeclosures)

@interact            
def clickShow(Effects=['None', 'Prediction Data', 'Clear']):
        if 'Geo' in layers:
            if Effects == 'Prediction Data':
                basicMapGeoData.on_click(on_click)
        
            
basicMap

NameError: name 'json' is not defined